In [ ]:
def validar_metas_asc(info):
    resultado = {
        "vacios": "",
        "umbrales": "",
        "metas": "",
        "mensaje_final": "",
        "patron_metas": ""
    }

    #Ver si está vacío
    if info.empty:
        resultado["mensaje_final"] = "No hay datos para validar."
        return resultado

    fila = info.iloc[0]

    #pasar las metas a datos numéricos
    metas_cols = [f"META{i}" for i in range(1, 13)]
    metas = [pd.to_numeric(fila.get(col), errors='coerce') for col in metas_cols]

    #Pasar la ifno que usaremos a numerico
    ciclo_base = pd.to_numeric(fila.get('CICLO_LINEA_BASE'), errors='coerce')
    linea_base = pd.to_numeric(fila.get('LINEA_BASE'), errors='coerce')
    umbral_verde = pd.to_numeric(fila.get('UMBRAL_VERDE'), errors='coerce')
    umbral_rojo = pd.to_numeric(fila.get('UMBRAL_ROJO'), errors='coerce')
    meta_anual = pd.to_numeric(fila.get("META12"), errors='coerce')

    no_12 = pd.isna(meta_anual)
    frecuencia = fila.get("FRECUENCIA_MEDICION")

    #1. Guardar la info referente a las metas que está vacía
    resultado["vacios"] = (
        f"{'Frecuencia no definida; ' if pd.isna(frecuencia) else ''}"
        f"{'El ciclo de la línea base no está definido; ' if pd.isna(ciclo_base) else ''}"
        f"{'La línea base no está definida; ' if pd.isna(linea_base) else ''}"
        f"{'Umbral verde no definido; ' if pd.isna(umbral_verde) else ''}"
        f"{'Umbral rojo no definido; ' if pd.isna(umbral_rojo) else ''}"
        f"{'Meta anual no definida; ' if no_12 else ''}"
    )

    #2. Validar los umbrales
    if (not pd.isna(umbral_rojo) and
        not pd.isna(umbral_verde) and
        not pd.isna(meta_anual)):

        if umbral_verde > umbral_rojo:
            if meta_anual >= linea_base:
                resultado["umbrales"] = "La meta anual coincide con la línea base coinciden con el sentido; "
            else:
                resultado["umbrales"] = "La meta anual debe ser mayor o igual a la línea base; "
        else:
            resultado["umbrales"] = "El umbral verde debe ser mayor al rojo; "
    else:
        resultado["umbrales"] = ""

    #Aquí se generan los errores según la frecuencia y si hay metas que no coinciden con el sentido registrado. Ver si la meta anual (12) es mayor o igual que la línea base
    def generar_errores(indices, frecuencia_reg):
        mensajes = []
        for i in reversed(range(1, len(indices))):
            idx_actual = indices[i] - 1
            idx_anterior = indices[i-1] - 1
            meta_actual = metas[idx_actual]
            meta_anterior = metas[idx_anterior]
            if not pd.isna(meta_actual) and not pd.isna(meta_anterior) and meta_actual < meta_anterior:
                mensajes.append(f"META {indices[i]} debe ser mayor o igual que META {indices[i-1]}; ")
        if len(indices) > 0:
            first_meta_idx = indices[0] - 1
            if not pd.isna(metas[first_meta_idx]) and not pd.isna(linea_base) and metas[first_meta_idx] < linea_base:
                mensajes.append(f"META {indices[0]} debe ser mayor o igual que la línea base; ")
        return "".join(mensajes)

    #Aquí se ve si es acumulado, constante o mal determinado
    def determinar_patron(indices):
        valid_metas = [metas[i-1] for i in indices if not pd.isna(metas[i-1])]
        if len(valid_metas) < 1:
            return "No hay suficientes metas para saber si es acumulado o constante"
        diferencias = [valid_metas[i] - valid_metas[i-1] for i in range(1, len(valid_metas))]

        if all(d > 0 for d in diferencias):
            return "Acumulado"
        if all(d == 0 for d in diferencias):
            return "Constante"
        if all(d < 0 for d in diferencias):
            return "Sentido mal determinado"

        return "No se pudo determinar el sentido"

    #Aquí se ve se ve si las metas son cocnsistentes para lkas frecuencias bianuales y trianual
    def frecuencia_especial(metas_esperadas):
        metas_no_na = [m for m in metas if not pd.isna(m)]

        if len(metas_no_na) != metas_esperadas:
            return {"error": f"Se requieren exactamente {metas_esperadas} metas para frecuencia {frecuencia} (encontradas: {len(metas_no_na)}); "}
        errores = []
        patron = None
        if metas_esperadas == 2:
            meta1, meta2 = metas_no_na[0], metas_no_na[1]

            if meta1 > meta2:
                patron = "Sentido mal determinado"
                errores.append("META1 debe ser menor o igual que la META2; ")
            elif meta1 == meta2:
                patron = "Constante"
            else:
                patron = "Acumulado"

            if not pd.isna(linea_base) and meta1 < linea_base:
                errores.append("META1 debe ser mayor o igual que la línea base; ")
        elif metas_esperadas == 3:
            meta1, meta2, meta3 = metas_no_na[0], metas_no_na[1], metas_no_na[2]

            if meta1 > meta2 or meta2 > meta3:
                if meta1 > meta2 and meta2 > meta3:
                    patron = "Sentido mal determinado"
                elif meta1 == meta2 and meta2 == meta3:
                    patron = "Constante"
                elif meta1 < meta2 and meta2 < meta3:
                    patron = "Acumulado"
                else:
                    patron = "No se pudo determinar el sentido"

                if meta1 > meta2:
                    errores.append("META1 debe ser menor o igual que la META2; ")
                if meta2 > meta3:
                    errores.append("META2 debe ser menor o igual que la META3; ")
            else:
                patron = "Acumulado"

            if not pd.isna(linea_base) and meta1 < linea_base:
                errores.append("META1 debe ser mayor o igual que la línea base; ")

        if not errores:
            errores.append(f"Las metas coinciden con el sentido {patron.lower()} y línea base; ")

        return {"error": "".join(errores), "patron": patron}

    #Aquí se usan las funciones definidas
    if frecuencia == "Bianual":
        validacion = frecuencia_especial(2)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    elif frecuencia == "Trianual":
        validacion = frecuencia_especial(3)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    else:
      #Aquí es poara obtener el índice de donde están las metas correspondientes a la frecuecnia
        indices_map = {
            "Mensual": list(range(1, 13)),
            "Bimestral": [2, 4, 6, 8, 10, 12],
            "Trimestral": [3, 6, 9, 12],
            "Cuatrimestral": [4, 8, 12],
            "Semestral": [6, 12],
            "Anual": [12],
            "Bienal": [12],
            "Trienal": [12],
            "Quinquenal": [12],
            "Sexenal": [12]
        }
        #Aquí es para saber cuantas debe haber vacías según la frecuencia registrada

        umbral_vacias_map = {
            "Mensual": 0,
            "Bimestral": 6,
            "Trimestral": 8,
            "Cuatrimestral": 9,
            "Semestral": 10,
            "Anual": 11,
            "Bienal": 11,
            "Trienal": 11,
            "Quinquenal": 11,
            "Sexenal": 11
        }

        indices = indices_map.get(frecuencia)
        umbral_vacias = umbral_vacias_map.get(frecuencia, 12)

        if indices is not None:
            err = generar_errores(indices, frecuencia)
            current_metas_for_freq = [metas[i-1] for i in indices]
            if sum(1 for m in current_metas_for_freq if pd.isna(m)) > umbral_vacias:
                resultado["metas"] = "Sin suficientes metas registradas; "
            elif not err:
                resultado["metas"] = "Las metas coinciden con el sentido y línea base; "
            else:
                resultado["metas"] = err

            patron = determinar_patron(indices)
            resultado["patron_metas"] = patron if patron is not None else ""

            if no_12 and frecuencia in ["Anual", "Bienal", "Trienal", "Quinquenal", "Sexenal"]:
                resultado["patron_metas"] = "No es posible determinar el sentido porque falta la meta anual"
        else:
            resultado["metas"] = "Frecuencia de medición no reconocida; "

    #para los mensajes sobre la validación de metas
    resultado["mensaje_final"] = (
        resultado["vacios"] +
        resultado["umbrales"] +
        resultado["metas"] +
        (resultado["patron_metas"] if resultado["patron_metas"] else "")
    )

    return resultado

In [ ]:
def validar_metas_desc(info):
    resultado = {
        "vacios": "",
        "umbrales": "",
        "metas": "",
        "mensaje_final": "",
        "patron_metas": ""
    }

    if info.empty:
        resultado["mensaje_final"] = "No hay datos para validar."
        return resultado

    fila = info.iloc[0]

    metas_cols = [f"META{i}" for i in range(1, 13)]
    metas = [pd.to_numeric(fila.get(col), errors='coerce') for col in metas_cols]
    ciclo_base = pd.to_numeric(fila.get('CICLO_LINEA_BASE'), errors='coerce')
    linea_base = pd.to_numeric(fila.get('LINEA_BASE'), errors='coerce')
    umbral_verde = pd.to_numeric(fila.get('UMBRAL_VERDE'), errors='coerce')
    umbral_rojo = pd.to_numeric(fila.get('UMBRAL_ROJO'), errors='coerce')
    meta_anual = pd.to_numeric(fila.get("META12"), errors='coerce')

    no_12 = pd.isna(meta_anual)
    frecuencia = fila.get("FRECUENCIA_MEDICION")

    resultado["vacios"] = (
        f"{'Frecuencia no definida; ' if pd.isna(frecuencia) else ''}"
        f"{'El ciclo de la línea base no está definido; ' if pd.isna(ciclo_base) else ''}"
        f"{'La línea base no está definida; ' if pd.isna(linea_base) else ''}"
        f"{'Umbral verde no definido; ' if pd.isna(umbral_verde) else ''}"
        f"{'Umbral rojo no definido; ' if pd.isna(umbral_rojo) else ''}"
        f"{'Meta anual no definida; ' if no_12 else ''}"
    )

    if (not pd.isna(umbral_rojo) and
        not pd.isna(umbral_verde) and
        not pd.isna(meta_anual)):
        if umbral_verde < umbral_rojo:
            if meta_anual <= linea_base:
                resultado["umbrales"] = "Los umbrales y la línea base coinciden con el sentido; "
            else:
                resultado["umbrales"] = "La meta anual debe ser menor o igual a la línea base; "
        else:
            resultado["umbrales"] = "El umbral verde debe ser menor al rojo; "
    else:
        resultado["umbrales"] = ""

    def generar_errores(indices, frecuencia_reg):
        mensajes = []

        for i in reversed(range(1, len(indices))):
            idx_actual = indices[i] - 1
            idx_anterior = indices[i-1] - 1

            meta_actual = metas[idx_actual]
            meta_anterior = metas[idx_anterior]

            if not pd.isna(meta_actual) and not pd.isna(meta_anterior) and meta_actual > meta_anterior:
                mensajes.append(f"META {indices[i]} debe ser menor o igual que META {indices[i-1]}; ")

        if len(indices) > 0:
            first_meta_idx = indices[0] - 1
            if not pd.isna(metas[first_meta_idx]) and not pd.isna(linea_base) and metas[first_meta_idx] > linea_base:
                mensajes.append(f"META {indices[0]} debe ser menor o igual que la línea base; ")

        return "".join(mensajes)

    def determinar_patron(indices):
        valid_metas = [metas[i-1] for i in indices if not pd.isna(metas[i-1])]
        if len(valid_metas) < 1:
            return None
        diferencias = [valid_metas[i] - valid_metas[i-1] for i in range(1, len(valid_metas))]
        if all(d < 0 for d in diferencias):
            return "Acumulado"
        if all(d == 0 for d in diferencias):
            return "Constante"
        if all(d > 0 for d in diferencias):
            return "Sentido mal determinado"
        return "No se pudo determinar el sentido"


    def frecuencia_especial(metas_esperadas):
        metas_no_na = [m for m in metas if not pd.isna(m)]
        if len(metas_no_na) != metas_esperadas:
            return {"error": f"Se requieren exactamente {metas_esperadas} metas para frecuencia {frecuencia} (encontradas: {len(metas_no_na)}); "}
        errores = []
        patron = None
        if metas_esperadas == 2:
            meta1, meta2 = metas_no_na[0], metas_no_na[1]
            if meta1 < meta2:
                patron = "Sentido mal determinado"
                errores.append("META1 debe ser mayor o igual que la META2; ")
            elif meta1 == meta2:
                patron = "Constante"
            else:
                patron = "Acumulado"
            if not pd.isna(linea_base) and meta1 > linea_base:
                errores.append("META1 debe ser menor o igual que la línea base; ")

        elif metas_esperadas == 3:
            meta1, meta2, meta3 = metas_no_na[0], metas_no_na[1], metas_no_na[2]

            if meta1 < meta2 or meta2 < meta3:
                if meta1 < meta2 and meta2 < meta3:
                    patron = "Sentido mal determinado"
                elif meta1 == meta2 and meta2 == meta3:
                    patron = "Constante"
                elif meta1 > meta2 and meta2 > meta3:
                    patron = "Acumulado"
                else:
                    patron = "No se pudo determinar el sentido"
                if meta1 < meta2:
                    errores.append("META1 debe ser mayor o igual que la META2; ")
                if meta2 < meta3:
                    errores.append("META2 debe ser mayor o igual que la META3; ")
            else:
                patron = "Acumulado"
            if not pd.isna(linea_base) and meta1 > linea_base:
                errores.append("META1 debe ser menor o igual que la línea base; ")
        if not errores:
            errores.append(f"Las metas coinciden con el sentido {patron.lower()} y línea base; ")
        return {"error": "".join(errores), "patron": patron}

    if frecuencia == "Bianual":
        validacion = frecuencia_especial(2)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    elif frecuencia == "Trianual":
        validacion = frecuencia_especial(3)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    else:
        indices_map = {
            "Mensual": list(range(1, 13)),
            "Bimestral": [2, 4, 6, 8, 10, 12],
            "Trimestral": [3, 6, 9, 12],
            "Cuatrimestral": [4, 8, 12],
            "Semestral": [6, 12],
            "Anual": [12],
            "Bienal": [12],
            "Trienal": [12],
            "Quinquenal": [12],
            "Sexenal": [12]
        }
        umbral_vacias_map = {
            "Mensual": 0,
            "Bimestral": 6,
            "Trimestral": 8,
            "Cuatrimestral": 9,
            "Semestral": 10,
            "Anual": 11,
            "Bienal": 11,
            "Trienal": 11,
            "Quinquenal": 11,
            "Sexenal": 11
        }
        indices = indices_map.get(frecuencia)
        umbral_vacias = umbral_vacias_map.get(frecuencia, 12)

        if indices is not None:
            err = generar_errores(indices, frecuencia)
            current_metas_for_freq = [metas[i-1] for i in indices]
            if sum(1 for m in current_metas_for_freq if pd.isna(m)) > umbral_vacias:
                resultado["metas"] = "Sin suficientes metas registradas; "
            elif not err:
                resultado["metas"] = "Las metas coinciden con el sentido y línea base; "
            else:
                resultado["metas"] = err
            patron = determinar_patron(indices)
            resultado["patron_metas"] = patron if patron is not None else ""

            if no_12 and frecuencia in ["Anual", "Bienal", "Trienal", "Quinquenal", "Sexenal"]:
                resultado["patron_metas"] = "No es posible determinar el sentido porque falta la meta anual"
        else:
            resultado["metas"] = "Frecuencia de medición no reconocida; "

    resultado["mensaje_final"] = (
        resultado["vacios"] +
        resultado["umbrales"] +
        resultado["metas"] +
        (resultado["patron_metas"] if resultado["patron_metas"] else "")
    )

    return resultado